This notebook illustrate how to connect to remote service.

## `Aletheia` service

`aletheia` service provides nice interface to read/write memory systems composed of hyper-vectors.

Currently it provides the following operations: 

* `get`: get/retrieve chunks that meets certain criteria;

* `set`: set/write chunks with certain metadata;

The chunks can be terminal, representing non-divisible concepts.
In addition, we can produce composite chunk that are sequences of chunks (terminal or composite), set of chunks, etc.

For example, if we model the English alphabet as terminal chunks, then each English word can be modelled as sequence of alphabets.

NOTE this requires `alatheia` service, either running at local machine or an Internet-accessible IP.


In [10]:
from kongming import api, data, memory

m = memory.RemoteMemory(target="localhost:8080")
with m.session():
    data.populate_ascii(m)
    data.populate_words(m, "en", [
        "the", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog",
        "world", "catherine", "university", "rate", "kitchen",
        "something", "activity", "necessary", "other", "stuff",
        "pattern", "director", "dying", "chinese",
        "specialize", "some", "for", "cat",
        "relax", "ing", "lifetime",
        "uzbekistan",
        "statistical"])

The script above connects to `aletheia` service, write English alphabets, followed by constructing some English words (as sequences of alphabets).

Some simple queries:

In [11]:
with m.session():
    got = m.get(api.CodePicker(
        ref=api.ChunkReference(namespace="en", token="relax")),
        session=api.Session(keep_semantics=True))
    print(got)

[category: SEQUENCE
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 3978420138648264203
  sparse_constrained {
    seed_word: "en:relax"
  }
}
semantic {
  hint: KNOT
  model: MODEL_1M_10BIT
  stable_hash: 8437652551019954451
  knot {
    parts {
      hint: SPARSE_CONSTRAINED
      sparse_constrained {
        prewired: SEQUENCE_START
      }
    }
    parts {
      hint: NECKLACE
      necklace {
        seed: 7115195808010466776
        pearls {
          hint: SEQUENCE
          members {
            hint: SPARSE_CONSTRAINED
            sparse_constrained {
              seed_word: "cp:r"
            }
          }
          members {
            hint: KNOT
            knot {
              parts {
                hint: SPARSE_CONSTRAINED
                sparse_constrained {
                  seed_word: "cp:e"
                }
              }
              parts {
                hint: SPARSE_CONSTRAINED
                sparse_constrained {
            

The output explains how the sequence of `relax` were constructed in the memory.
First it's a `SEQUENCE` made of `r`, `e`, `l`, `a`, and `x` from namespace of `unicode`.
Its semantic code is the combination of algebraic operation of both binding (knot) and bundling (necklace).

Next example illustrates how we can recover individual member from a composite vector.

In [12]:
with m.session():
    wordPicker = api.CodePicker(
        ref=api.ChunkReference(namespace="en", token="pattern"))
    got = m.get(api.CodePicker(
        sequence_members=api.SequenceMembersPicker(sequence=wordPicker)))
    print(got)

[category: TERMINAL
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 12734596041969989932
  sparse_constrained {
    seed_word: "cp:p"
  }
}
refs {
  namespace: "cp"
  token: "p"
}
weight: 157
, category: TERMINAL
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 4290438932262761189
  sparse_constrained {
    seed_word: "cp:a"
  }
}
refs {
  namespace: "cp"
  token: "a"
}
weight: 169
, category: TERMINAL
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 17360189598024595871
  sparse_constrained {
    seed_word: "cp:t"
  }
}
refs {
  namespace: "cp"
  token: "t"
}
weight: 121
, category: TERMINAL
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 17360189598024595871
  sparse_constrained {
    seed_word: "cp:t"
  }
}
refs {
  namespace: "cp"
  token: "t"
}
weight: 159
, category: TERMINAL
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 94037985245

In this case, with a simple composite code for word `pattern`, the memory can recover its members: `p`, `a`, `t`, `t`, `e`, `r`, `n`, with each taking a weight/overlap of roughly 1/7 of the total 1024 ON bits.

In addition, we can retrieve only a subset of sequence members, starting from position 2 (0-based) and up to 3 members.

In [13]:
with m.session():
    wordPicker = api.CodePicker(
        ref=api.ChunkReference(namespace="en", token="pattern"))
    got = m.get(api.CodePicker(
        sequence_members=api.SequenceMembersPicker(sequence=wordPicker, start=2, limit=3)))
    print(got)

[category: TERMINAL
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 17360189598024595871
  sparse_constrained {
    seed_word: "cp:t"
  }
}
refs {
  namespace: "cp"
  token: "t"
}
weight: 121
, category: TERMINAL
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 17360189598024595871
  sparse_constrained {
    seed_word: "cp:t"
  }
}
refs {
  namespace: "cp"
  token: "t"
}
weight: 159
, category: TERMINAL
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 9403798524593381096
  sparse_constrained {
    seed_word: "cp:e"
  }
}
refs {
  namespace: "cp"
  token: "e"
}
weight: 143
]


Fuzzy query can be peformed like this:

In [14]:
with m.session():
    wordPicker = api.CodePicker(
        ref=api.ChunkReference(namespace="en", token="world"))
    got = m.get(api.CodePicker(near_neighbor=wordPicker))
    print(got)

[category: SEQUENCE
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 14026031177565144867
  sparse_constrained {
    seed_word: "en:world"
  }
}
refs {
  namespace: "en"
  token: "world"
}
weight: 1024
, category: SEQUENCE
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 8547901421001462119
  sparse_constrained {
    seed_word: "en:for"
  }
}
refs {
  namespace: "en"
  token: "for"
}
weight: 129
, category: SEQUENCE
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 5365463852431912213
  sparse_constrained {
    seed_word: "en:dog"
  }
}
refs {
  namespace: "en"
  token: "dog"
}
weight: 85
, category: SEQUENCE
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 11816919111036053690
  sparse_constrained {
    seed_word: "en:fox"
  }
}
refs {
  namespace: "en"
  token: "fox"
}
weight: 61
, category: SEQUENCE
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable

This result is worth careful examination and discussion.

* First of all, the word `world` itself is returned (as full match, overlap=1024);
* the word `for` is returned as second, with weight/overlap=129, since there are two letters overlap: `o` and `r`;
* the word `dog` is returned, with weight/overlap=85, 1 letter overlapping: `o`;
* the word `fox` is returned, with weight/overlap=61, 1 letter overlapping: `o`;
* the word `some` is returned, with weight/overlap=54, 1 letter overlapping: `o`;
* the word `director` is returned, with weight/overlap=28, 1 letter overlapping: `r`;
* the word `something` is returned, with weight/overlap=21, 1 letter overlappng: `o`.

All fuzzy matching is done at the server side, modelling cognitive similarity with the query word.
Currently the matching can be achieved within a few milli-seconds.

In addition, optional `session` can be used to control what range of results to return.

In [15]:
with m.session():
    wordPicker = api.CodePicker(
        ref=api.ChunkReference(namespace="en", token="world"))
    got = m.get(api.CodePicker(near_neighbor=wordPicker), session=api.Session(start=0, limit=4))
    print(got)

[category: SEQUENCE
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 14026031177565144867
  sparse_constrained {
    seed_word: "en:world"
  }
}
refs {
  namespace: "en"
  token: "world"
}
weight: 1024
, category: SEQUENCE
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 8547901421001462119
  sparse_constrained {
    seed_word: "en:for"
  }
}
refs {
  namespace: "en"
  token: "for"
}
weight: 129
, category: SEQUENCE
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 5365463852431912213
  sparse_constrained {
    seed_word: "en:dog"
  }
}
refs {
  namespace: "en"
  token: "dog"
}
weight: 85
, category: SEQUENCE
elemental {
  hint: SPARSE_CONSTRAINED
  model: MODEL_1M_10BIT
  stable_hash: 11816919111036053690
  sparse_constrained {
    seed_word: "en:fox"
  }
}
refs {
  namespace: "en"
  token: "fox"
}
weight: 61
]
